<a href="https://colab.research.google.com/github/j03m/lstm-price-predictor/blob/main/Coin_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Current Todo:


Some ideas here: 

* Use pandas_ta to attach technical data to timeseries then use random forest to see if any features are super good.

* Use vector_bt to back test your models

* Integrate the different time granularity models, 15m etc into the core configs

* Fix coinbase - we can't filter on volume amount without extra work

* Add TA fields, use random forest to check which fields are the best, verify with mean square error (this could take a while)

* Move to an "always" on model where we more closely monitor exits and entries. We can constantly check predictions and refine entry/exit points. Can we pick a better entry point? We had at some point discussed trying to predict all 4 values which would help. 

* Can we monitor and alarm/email the 15 min chart of an entry and see if it is
move toward or away from our target?

* pandas_ta strategy/back testers look interesting?

Read me: https://www.kaggle.com/code/vuhuyduongnia/vn30-stock-prediction-by-lstm-model-accuracy-90


#IMPORT DATASETS AND LIBRARIES


In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import sys

sys.path.insert(0,'/content/drive/My Drive/ml-trde-notebooks')




Mounted at /content/drive


# Library

In [3]:
# ***** WARNING : Install deps - This will BUILD TALib and takes a while!
%run -i '/content/drive/My Drive/ml-trde-notebooks/installs.ipynb'

--2023-01-22 13:34:26--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2023-01-22 13:34:26--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://onboardcloud.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2023-01-22 13:34:26--  http://onboardcloud.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving onboa

In [4]:
# Executes this notebook in our space, making all of its functions/globals available
%run -i '/content/drive/My Drive/ml-trde-notebooks/common.ipynb'

Modified
1674346972
1674346972.0
1674346972.0


/usr/local/lib/python3.8/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [5]:
models = {}

# Data Loader

In [6]:
def load_data():
  #we don't need 15 right now because they get loaded in the train, that will prob need to be more sophisticated soon
  #all_stock_df_15 = []
  for ticker in tickers:
    print("load:", ticker)
    df = sort_date(pd.read_csv(data_path +'/' + ticker + '.csv'))
    df["Date"] = pd.to_datetime(df['Date'], utc=True)
    all_stock_dfs.append(df)
    #df2 = sort_date(pd.read_csv(data_path + '/' + ticker + '-15.csv').rename(columns={"Datetime":"Date"}))
  print("all dfs:", len(all_stock_dfs))
  #for coin in coins:
  #  print("load:", coin)
  #  df = sort_date(pd.read_csv(data_path +'/' + coin + '.csv'))
  #  all_coin_dfs.append(df)





# Training/Loading

In [7]:
build_and_stash_all_config_models()

build model: lstm_cv  features: 2
build model: lstm_coins_cv  features: 2
build model: lstm_att_cv  features: 2
build model: lstm_att_ohlcv  features: 5
build model: lstm_cv_rvi  features: 3
build model: lstm_cv_vwap  features: 4
build model: lstm_ohlc  features: 5
build model: svm_cv  features: 2
build model: svm_cv_vwap  features: 4
build model: lstm_xgb_cols  features: 9


## Load all models from disk

In [8]:
if load_models:
  training_filter = model_config["training_filter"]
  for name in model_config["day_bar_models"]:
    if (name in training_filter): #skip training models
      print("skipping: ", name)
      continue
    print("loading:", name)
    if name in model_config["load_type"] and model_config["load_type"][name] == "joblib":
      models[name] = joblib.load(model_path + "/" + name  + "-" + model_version_token + ".joblib") 
    else:
      models[name] = keras.models.load_model(model_path + "/" + name + "-" + model_version_token + ".h15")
  lstm_15m = keras.models.load_model(model_path + "/lstm_15m.h15")
  print ("models loaded")  

else:
  print ("Not loading")
  


Not loading


## Train and Save Models


In [9]:
histories = {}
training_filter = model_config["training_filter"]
if train_models:
  load_data()
  for name in model_config["day_bar_models"]:
    if (len(training_filter)!=0 and not name in training_filter):
      print("skipping: ", name)
      continue
    build_model_from_config(models, name, model_config)
    print("training: ", name)
    # todo - we need a training set type flag in config for 15m models to get the correct files/data
    training_dfs = get_training_datasets_for_model(name)
    print("training dfs:", len(training_dfs))
    count = 0
    for df in training_dfs:
      print("training: ", name, " df: ", count)
      count+=1
      history = train_config_model_against_df(name, model_config, df)
      if not name in histories:
        histories[name] = []
      histories[name].append(history)
    if save_models:
      print("saving:", name)
      if name in model_config["load_type"] and model_config["load_type"][name] == "joblib":
        joblib.dump(models[name], model_path + "/" + name  + "-" + model_version_token + ".joblib") 
      else:
        models[name].save(model_path + "/" + name  + "-" + model_version_token + ".h15")
  lstm_15m = build_15m_model()
  lstm_15m.save(model_path + "/lstm_15m-2.h15")

load: SPY
load: TSLA
load: AAPL
load: IBM
load: F
load: CAT
load: BAC
load: B
load: META
load: AMZN
load: XOM
load: BP
load: CHK
load: GME
load: MRNA
load: BA
load: PG
load: NEE
load: FCX
load: Z
load: NVDA
load: PFE
load: WMT
load: NOK
load: T
load: BABA
load: AMC
load: SQ
load: SCCO
load: GOOGL
load: GOOG
load: GE
load: CVX
load: CSCO
load: CMCSA
load: CL
load: CBRE
load: CB
load: DAL
load: D
load: EBAY
load: EBAY
load: EMR
load: PBF
load: NEM
load: FCX
load: AA
load: VZ
load: T
load: DVN
load: CHK
load: HIMX
load: AMD
load: NXPI
load: PFE
all dfs: 55
build model: lstm_cv  features: 2
training:  lstm_cv
lstm_cv  should train on stocks
training dfs: 55
training:  lstm_cv  df:  0
Training model: lstm_cv  features: 2


/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/datetimes.py:1143: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


Epoch 1/200
189/189 [==============================] - 11s 13ms/step - loss: 0.0052 - val_loss: 0.2855
Epoch 2/200
189/189 [==============================] - 1s 8ms/step - loss: 7.2425e-04 - val_loss: 0.2584
Epoch 3/200
189/189 [==============================] - 1s 8ms/step - loss: 5.6333e-04 - val_loss: 0.1672
Epoch 4/200
189/189 [==============================] - 1s 8ms/step - loss: 4.3237e-04 - val_loss: 0.0386
Epoch 5/200
189/189 [==============================] - 1s 8ms/step - loss: 3.4637e-04 - val_loss: 0.0137
Epoch 6/200
189/189 [==============================] - 1s 8ms/step - loss: 3.4349e-04 - val_loss: 0.0076
Epoch 7/200
189/189 [==============================] - 1s 8ms/step - loss: 3.5265e-04 - val_loss: 0.0080
Epoch 8/200
189/189 [==============================] - 1s 8ms/step - loss: 2.9893e-04 - val_loss: 0.0119
Epoch 9/200
189/189 [==============================] - 1s 8ms/step - loss: 3.1442e-04 - val_loss: 0.0134
Epoch 10/200
189/189 [==============================] - 1

/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/datetimes.py:1143: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


80/80 [==============================] - 1s 8ms/step - loss: 1.5155e-04 - val_loss: 0.0500
Epoch 2/200
80/80 [==============================] - 1s 8ms/step - loss: 9.6547e-05 - val_loss: 0.0208
Epoch 3/200
80/80 [==============================] - 1s 8ms/step - loss: 5.3378e-05 - val_loss: 0.0077
Epoch 4/200
80/80 [==============================] - 1s 8ms/step - loss: 5.4857e-05 - val_loss: 0.0056
Epoch 5/200
80/80 [==============================] - 1s 8ms/step - loss: 5.5979e-05 - val_loss: 0.0091
Epoch 6/200
80/80 [==============================] - 1s 8ms/step - loss: 1.6785e-04 - val_loss: 0.0055
Epoch 7/200
80/80 [==============================] - 1s 8ms/step - loss: 5.1634e-05 - val_loss: 0.0137
Epoch 8/200
80/80 [==============================] - 1s 8ms/step - loss: 3.9017e-05 - val_loss: 0.0144
Epoch 9/200
80/80 [==============================] - 1s 8ms/step - loss: 3.3588e-05 - val_loss: 0.0238
Epoch 10/200
80/80 [==============================] - 1s 8ms/step - loss: 2.7997e-05 

...

270/270 [==============================] - 2s 9ms/step - loss: 2.6658e-05 - val_loss: 2.2544e-04
Epoch 2/200
270/270 [==============================] - 2s 9ms/step - loss: 1.7219e-05 - val_loss: 2.2738e-04
Epoch 3/200
270/270 [==============================] - 2s 9ms/step - loss: 1.6002e-05 - val_loss: 2.2419e-04
Epoch 4/200
270/270 [==============================] - 2s 9ms/step - loss: 1.5342e-05 - val_loss: 2.3091e-04
Epoch 5/200
270/270 [==============================] - 2s 9ms/step - loss: 1.5696e-05 - val_loss: 2.3314e-04
Epoch 6/200
270/270 [==============================] - 2s 9ms/step - loss: 1.5700e-05 - val_loss: 2.0806e-04
Epoch 7/200
270/270 [==============================] - 2s 9ms/step - loss: 1.5534e-05 - val_loss: 2.1090e-04
Epoch 8/200
270/270 [==============================] - 2s 9ms/step - loss: 1.4809e-05 - val_loss: 2.4885e-04
Epoch 9/200
270/270 [==============================] - 2s 9ms/step - loss: 1.5677e-05 - val_loss: 2.2991e-04
Epoch 10/200
270/270 [=========

/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/datetimes.py:1143: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


79/79 [==============================] - 1s 9ms/step - loss: 9.6055e-05 - val_loss: 0.0012
Epoch 2/200
79/79 [==============================] - 1s 9ms/step - loss: 8.9230e-05 - val_loss: 0.0012
Epoch 3/200
79/79 [==============================] - 1s 9ms/step - loss: 8.0581e-05 - val_loss: 0.0013
Epoch 4/200
79/79 [==============================] - 1s 9ms/step - loss: 7.9036e-05 - val_loss: 0.0014
Epoch 5/200
79/79 [==============================] - 1s 9ms/step - loss: 7.8941e-05 - val_loss: 0.0012
Epoch 6/200
79/79 [==============================] - 1s 9ms/step - loss: 8.2409e-05 - val_loss: 0.0012
Epoch 7/200
79/79 [==============================] - 1s 9ms/step - loss: 7.8997e-05 - val_loss: 0.0013
Epoch 8/200
79/79 [==============================] - 1s 9ms/step - loss: 8.1547e-05 - val_loss: 0.0013
Epoch 9/200
79/79 [==============================] - 1s 9ms/step - loss: 7.6275e-05 - val_loss: 0.0012
Epoch 10/200
79/79 [==============================] - 1s 9ms/step - loss: 8.6069e-05 

/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/datetimes.py:1143: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


320/320 [==============================] - 3s 9ms/step - loss: 4.6849e-05 - val_loss: 4.1299e-04
Epoch 2/200
320/320 [==============================] - 3s 9ms/step - loss: 4.7026e-05 - val_loss: 3.2438e-04
Epoch 3/200
320/320 [==============================] - 3s 8ms/step - loss: 4.4724e-05 - val_loss: 3.8195e-04
Epoch 4/200
320/320 [==============================] - 3s 9ms/step - loss: 4.4031e-05 - val_loss: 3.5126e-04
Epoch 5/200
320/320 [==============================] - 3s 9ms/step - loss: 4.2434e-05 - val_loss: 3.3333e-04
Epoch 6/200
320/320 [==============================] - 3s 9ms/step - loss: 4.3193e-05 - val_loss: 3.1646e-04
Epoch 7/200
320/320 [==============================] - 3s 9ms/step - loss: 4.2647e-05 - val_loss: 3.0336e-04
Epoch 8/200
320/320 [==============================] - 3s 8ms/step - loss: 4.3395e-05 - val_loss: 3.2321e-04
Epoch 9/200
320/320 [==============================] - 3s 9ms/step - loss: 4.5770e-05 - val_loss: 3.2575e-04
Epoch 10/200
320/320 [=========

## Train a single model against a single dataset

In [10]:
#build_and_stash_all_config_models()
#coin_dfs = get_raw_data_for_coin_list(180)

In [11]:
#[model, history] = train_config_model_against_df("lstm_cv", model_config, coin_dfs["FCON-USDT"])
#models["lstm_cv"] = model
#df_raw = get_coin_data_frames(180, "FCON-USDT")
#test_name = "lstm_ohlc"
#all_columns = model_config["column_sets"][test_name]+gbl_target_column
#[scaled_features, X, y, normal_features] = convert_to_training_dataset(df, columns=all_columns)  
#[p_scaled, p] = predict_trade(models[test_name], test_name, "FCON-USDT", X, all_columns)
#build_trade_model(p, normal_features, "FCON-USDT", test_name)

# Save all models to disk

In [ ]:
if save_models:
  for name in model_config["day_bar_models"]:
    if (len(training_filter)!=0 and not name in training_filter):
      print("skipping: ", name)
      continue
    print("saving:", name)
    if name in model_config["load_type"] and model_config["load_type"][name] == "joblib":
      joblib.dump(models[name], model_path + "/" + name  + "-" + model_version_token + ".joblib") 
    else:
      models[name].save(model_path + "/" + name  + "-" + model_version_token + ".h15")
 
  print ("models saved")  
else:
  print ("Not saving")


# Visualize and Compare all Models

In [13]:
df_raw = get_coin_data_frames(1500, "BTC-USDT")

In [ ]:
backtest_filter = []
[results, data, fig] = renderPredictions(df_raw, models, backtest_filter, False, False)
results

In [15]:
df_raw

,Date,Open,Close,High,Low,Volume,amount
0,2023-01-22,22780.900000,22805.200000,23071.800000,22609.800000,6604.317632,150918778.450203
1,2023-01-21,22668.800000,22780.900000,23369.900000,22417.000000,10272.731368,235458722.819069
2,2023-01-20,21073.600000,22668.700000,22748.200000,20866.000000,9876.527761,212945047.854365
3,2023-01-19,20676.700000,21073.500000,21199.200000,20659.600000,6165.239500,128704497.315423
4,2023-01-18,21131.800000,20676.700000,21639.200000,20404.000000,9782.348212,206164413.013461
...,...,...,...,...,...,...,...
1495,2018-12-19,3651.644898,3664.467780,3899.965240,3623.959411,534.193179,1991870.358672
1496,2018-12-18,3509.804799,3651.644898,3669.542359,3438.437007,314.954644,1112127.961140
1497,2018-12-17,3226.444891,3511.733273,3581.900000,3218.550073,494.875964,1704568.758381
1498,2018-12-16,3214.032737,3229.359252,3285.806038,3210.745450,223.992961,727775.558003


# What has a buy indicator for tomorrow?

In [ ]:
# Fetch the top 10 and see if they predict up
df_products = get_all_products()
df_products = df_products[df_products.id.str.endswith('USDT')]

report_columns = ["Product",
                  "Model Name",
                  "Close",
                  "Target",
                  "Predicted",
                  "MSE",
                  "Percent",
                  "250Fees",
                  "5kFees",
                  "10kFees",
                  "250Profit",
                  "5kProfit",
                  "10k0Profit",
                  "Period"]

# we have a desired budget of 10k in trading capital to deploy. 
# "volValue": is the 24h total, the trading volume in quote currency of last 24 hours
# Any markets we enter need to have a signifcantly higher trading value volume otherwise
# we can't really trade there without breaking things up. As we grow our strat here will need to change.
# for now we cut down our set to 1m in volVal
volumeCutoff = 500000
df_products["volValue"] = [float(x) for x in df_products['volValue']]
df_products = df_products[df_products["volValue"] > volumeCutoff]
df_products = df_products.sort_values(by = ['id'])

models_in_play = [
        "lstm_cv", 
        "lstm_att_cv", 
        "lstm_cv_rvi",
        "svm_cv",
        "lstm_xgb_cols"]
if coin_base:
  df_products = df_products[df_products.trading_disabled == False]
  df_products = df_products[df_products.cancel_only == False]

df_trades = pd.DataFrame();
df_estc = pd.DataFrame(); #expected short term closes
df_estc["Product"] = [];
df_estc["Est Close"] = [];
df_estc["Est Close Raw"] = [];
bars_long = 180
bars_short = 30
counter = 0;
print("iterating through:", len(df_products))
tries = 3

def downloadAndPredict(all_trades, product, length):
  print("download day bars: ", product, " bar set:", length)
  df_raw = get_coin_data_frames(length, product)
  for name in models_in_play:
    print("predicting trade:", name, "for", product)
    df_trade = predict_config_model_for_product(df_raw, name, product)
    df_trade["Period"] = [length]
    df_trade = df_trade[report_columns]
    print("predicted mse:", name, ":{:.10f}".format(df_trade["MSE"].iloc[0]))
    all_trades = all_trades.append(df_trade)
  return all_trades

def downloadAndPredict15(all_trades, product):
  print("download 15 bars: ", product)
  [predicted_scaled, predicted] = fetch_and_predict_short_term(lstm_15m,product)
  df2 = pd.DataFrame({'Product': [product], 'Est Close': [predicted], 'Est Close Raw': predicted_scaled})
  all_trades = all_trades.append(df2)
  return all_trades


for index, row in df_products.iterrows():
    
    loop = True
    count = 0
    while(loop):
      try:
        print("start day long")
        df_trades = downloadAndPredict(df_trades, row.id, bars_long)
        time.sleep(1)
        
        print("start day short")
        df_trades = downloadAndPredict(df_trades, row.id, bars_short)
        time.sleep(1)

        print("start 15m")
        df_estc = downloadAndPredict15(df_estc, row.id)
        time.sleep(1)
        loop = False
      except Exception as inst:
        #raise inst
        print("Error: ", inst)
        time.sleep(1)
        count = count+1
        if count>tries:
          loop = False

df_trades.reset_index()
df_trades_filtered = df_trades  # TODO: Filter again after see what the new mse's look like #df_trades.loc[(df_trades["Period"] == 180) & (df_trades["MSE"] < 0.00005) | (df_trades["Period"] == 30) & (df_trades["MSE"] < 0.0005)]
df_buys = df_trades_filtered[df_trades_filtered['Percent'] > 0] 
df_shorts = df_trades_filtered[df_trades_filtered['Percent'] < 0] 
df_weighted = df_trades_filtered.groupby("Product").apply(consensus_prediction)
df_weighted = df_weighted.rename("Consensus Prediction")
df_trades_final = pd.merge(df_trades_filtered, df_weighted, left_on='Product', right_index=True)
df_trades_final["Consensus Percent"] = ((df_trades_final['Consensus Prediction'] - df_trades_final['Close']) / df_trades_final['Close']) * 100
df_view = df_trades_final[["Product",  "Model Name",  "MSE", "Period", "Close", "Predicted", "Percent","Consensus Percent", "Consensus Prediction"]]
df_view = df_view.sort_values(by=["Consensus Percent", "MSE"], ascending=[True, False])

In [36]:

df_view

,Product,Model Name,MSE,Period,Close,Predicted,Percent,Consensus Percent,Consensus Prediction
Date,,,,,,,,,
2023-01-21,OCEAN-USDT,lstm_cv_rvi,0.001496,30,0.319500,0.258777,-19.005737,-16.028653,0.268288
2023-01-21,OCEAN-USDT,lstm_xgb_cols,0.001468,30,0.319500,0.264971,-17.066827,-16.028653,0.268288
2023-01-21,OCEAN-USDT,lstm_att_cv,0.001002,30,0.319500,0.263091,-17.655315,-16.028653,0.268288
2023-01-21,OCEAN-USDT,lstm_cv,0.000829,30,0.319500,0.266081,-16.719425,-16.028653,0.268288
2023-01-21,OCEAN-USDT,lstm_xgb_cols,0.000449,180,0.319500,0.259871,-18.663281,-16.028653,0.268288
...,...,...,...,...,...,...,...,...,...
2023-01-21,BOND-USDT,lstm_cv_rvi,0.000325,180,5.122000,7.048556,37.613346,13.771915,5.827397
2023-01-21,BOND-USDT,lstm_att_cv,0.000188,180,5.122000,7.088149,38.386350,13.771915,5.827397
2023-01-21,BOND-USDT,lstm_cv,0.000136,180,5.122000,7.112312,38.858100,13.771915,5.827397


In [29]:
df_view

,Product,Model Name,MSE,Period,Close,Predicted,Percent,Consensus Percent,Consensus Prediction
Date,,,,,,,,,
2023-01-21,OCEAN-USDT,lstm_cv_rvi,0.001496,30,0.319500,0.258777,-19.005737,-16.028653,0.268288
2023-01-21,OCEAN-USDT,lstm_xgb_cols,0.001468,30,0.319500,0.264971,-17.066827,-16.028653,0.268288
2023-01-21,OCEAN-USDT,lstm_att_cv,0.001002,30,0.319500,0.263091,-17.655315,-16.028653,0.268288
2023-01-21,OCEAN-USDT,lstm_cv,0.000829,30,0.319500,0.266081,-16.719425,-16.028653,0.268288
2023-01-21,OCEAN-USDT,lstm_xgb_cols,0.000449,180,0.319500,0.259871,-18.663281,-16.028653,0.268288
...,...,...,...,...,...,...,...,...,...
2023-01-21,BOND-USDT,lstm_cv_rvi,0.000325,180,5.122000,7.048556,37.613346,13.771915,5.827397
2023-01-21,BOND-USDT,lstm_att_cv,0.000188,180,5.122000,7.088149,38.386350,13.771915,5.827397
2023-01-21,BOND-USDT,lstm_cv,0.000136,180,5.122000,7.112312,38.858100,13.771915,5.827397


In [19]:
df_estc

,Product,Est Close,Est Close Raw
0,AAVE-USDT,88.896239,0.723582
0,ACE-USDT,0.008610,0.673652
0,ADA-USDT,0.382799,0.728841
0,AGIX-USDT,0.197184,0.936661
0,ALBT-USDT,0.110593,0.575007
...,...,...,...
0,XMR-USDT,180.452771,0.883929
0,XPR-USDT,0.002061,0.746450
0,XRP-USDT,0.409741,0.876525
0,ZEC-USDT,48.545110,0.947574


In [20]:
now = datetime.now()
today = now.strftime("%Y-%m-%d")

if coin_base:
  token = "cb"
else:
  token = "ku"

df_buys.to_csv(data_path+"/buy-" + token + "-" + today + ".csv")
df_estc.to_csv(data_path+"/15m-" + token + "-" + today + ".csv")
df_shorts.to_csv(data_path+"/shorts-" + token + "-" + today + ".csv")
df_trades_final.to_csv(data_path+"/final-" + token + "-" + today + ".csv")
df_view.to_csv(data_path+"/view-" + token + "-" + today + ".csv")